In [2]:
from __future__ import division, print_function

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [58]:
import time
from collections import namedtuple

import pydrake.solvers.mathematicalprogram as mp
import numpy as np
from irispy import Polyhedron
from director import viewerclient as vc

from polynomial import Polynomial
from piecewise import Piecewise

In [69]:
vis = vc.Visualizer()

In [5]:
def piecewise_polynomial_variable(prog, domain, dimension, degree, kind="continuous"):
    if kind == "continuous":
        C = [prog.NewContinuousVariables(degree + 1, len(domain) - 1) for _ in range(dimension)]
    elif kind == "binary":
        C = [prog.NewBinaryVariables(degree + 1, len(domain) - 1) for _ in range(dimension)]
    else:
        raise ValueError("Expected kind to be 'continuous' or 'binary', but got {:s}".format(kind))    
    C = np.stack(C, axis=2)
    assert C.shape == (degree + 1, len(domain) - 1, dimension)
    return Piecewise(domain,
                     [Polynomial([C[i, j, :] for i in range(degree + 1)]) 
                          for j in range(len(domain) - 1)])

In [6]:
def require_continuity(prog, piecewise):
    for t in piecewise.breaks[1:-1]:
        frombelow = piecewise.from_below(t)
        fromabove = piecewise.from_above(t)
        for i in range(frombelow.size):
            prog.AddLinearConstraint(frombelow.flat[i].Expand() == fromabove.flat[i].Expand())
    

In [48]:
class BoxAtlas(object):
    dim = 2
    g = 9.81
    mass = 10
    limb_velocity_limits = [
        10,
        10,
        10,
        10
    ]
    
    limb_bounds = [
        Polyhedron.fromBounds([0.5, -0.5], [1.0, 0.5]),    # right arm
        Polyhedron.fromBounds([0.0, -1.0], [0.5, -0.5]),   # right leg
        Polyhedron.fromBounds([-0.5, -1.0], [0.0, -0.5]),  # left leg
        Polyhedron.fromBounds([-1.0, -0.5], [-0.5, 0.5])  # left arm
    ]
    
    def __init__(self):
        pass

    
class BoxAtlasState(object):
    def __init__(self, robot, qcom=None, vcom=None, qlimb=None):
        self.robot = robot
        if qcom is None:
            qcom = np.zeros(robot.dim)
        if vcom is None:
            vcom = np.zeros(robot.dim)
        if qlimb is None:
            qlimb = [np.zeros(robot.dim) for _ in robot.limb_bounds]
        
        self.qcom = qcom
        self.vcom = vcom
        self.qlimb = qlimb
            
    def draw(self, vis, atlasinput=None):
        vis["body"].setgeometry(vc.Box(lengths=[0.1, 0.1, 0.1]))
        vis["body"].settransform(vc.transformations.translation_matrix([self.qcom[0], 0, self.qcom[1]]))
        for (i, q) in enumerate(self.qlimb):
            v = vis["limb_{:d}".format(i)]
            v.setgeometry(vc.Sphere(radius=0.05))
            v.settransform(vc.transformations.translation_matrix([q[0], 0, q[1]]))
            
        
class BoxAtlasInput(object):
    def __init__(self, robot, flimb=None):
        self.robot = robot
        if flimb is None:
            flimb = [np.zeros(robot.dim) for _ in robot.limb_bounds]
        self.flimb = flimb
        
Surface = namedtuple("Surface", ["pose_constraints", "force_constraints"])
        
class Environment(object):
    def __init__(self, surfaces):
        self.surfaces = surfaces
        
class Trajectory(object):
    def __init__(self, components, constructor):
        self.components = components
        self.constructor = constructor
    
    def __call__(self, t):
        return self.constructor(*[c(t) for c in self.components])

In [49]:
def get_piecewise_solution(prog, piecewise):
    return piecewise.map(lambda p: p.map(lambda x: prog.GetSolution(x)))


In [109]:
def contact_stabilize(initial_state, env):
    robot = initial_state.robot
    dt = 0.01
    time_horizon = 20 * dt
    ts = np.linspace(0, time_horizon, time_horizon / dt + 1)
    dim = robot.dim
    num_limbs = len(robot.limb_bounds)
    gravity_force = np.zeros(dim)
    gravity_force[-1] = -robot.mass * robot.g
    vlimb_max = 5
    
    Mq = 10
    Mv = 100
    Mf = 1000

    prog = mp.MathematicalProgram()
    qcom = piecewise_polynomial_variable(prog, ts, dim, 2)
    require_continuity(prog, qcom)
    vcom = qcom.map(Polynomial.derivative)
    require_continuity(prog, vcom)
    acom = vcom.map(Polynomial.derivative)
    
    qlimb = [piecewise_polynomial_variable(prog, ts, dim, 0) for k in range(num_limbs)]
    contact_force = [piecewise_polynomial_variable(prog, ts, dim, 0) for k in range(num_limbs)]
    contact = [piecewise_polynomial_variable(prog, ts, dim, 0, kind="binary") for k in range(num_limbs)]

    for k in range(num_limbs):
        for j in range(len(ts) - 1):
            t = ts[j]
            indicator = contact[k](t)[0]
            A = env.surfaces[k].pose_constraints.getA()
            b = env.surfaces[k].pose_constraints.getB()
            qlimb_after_dt = qlimb[k].from_below(ts[j + 1])
            for i in range(A.shape[0]):
                prog.AddLinearConstraint((A[i, :].dot(qlimb_after_dt) - (b[i] + Mq * (1 - indicator))).Expand() <= 0)
            
            Af = env.surfaces[k].force_constraints.getA()
            bf = env.surfaces[k].force_constraints.getB()
            for i in range(Af.shape[0]):
                prog.AddLinearConstraint((Af[i, :].dot(contact_force[k](t)) - (b[i] + Mf * (1 - indicator))).Expand() <= 0)
            
            for i in range(dim):
                prog.AddLinearConstraint((contact_force[k](t)[i] - (Mf * indicator)).Expand() <= 0)
            
            if j < len(ts) - 3:
                for i in range(dim):
                    prog.AddLinearConstraint(((qlimb[k](ts[j + 1])[i] - qlimb[k](ts[j + 2])[i]) - (Mv * (1 - indicator))).Expand() <= 0)
                    prog.AddLinearConstraint((-1 * (qlimb[k](ts[j + 1])[i] - qlimb[k](ts[j + 2])[i]) - (Mv * (1 - indicator))).Expand() <= 0)
            
    for k in range(num_limbs):
        for j in range(len(ts) - 2):
            t = ts[j]
            for i in range(dim):
                prog.AddLinearConstraint(((qlimb[k](ts[j + 1])[i] - qlimb[k](t)[i]) - vlimb_max).Expand() <= 0)
                prog.AddLinearConstraint((-1 * (qlimb[k](ts[j + 1])[i] - qlimb[k](t)[i]) - vlimb_max).Expand() <= 0)
        
    # TODO: hard-coded free space
    for t in ts[:-1]:
        for i in range(dim):
            prog.AddLinearConstraint(qcom(t)[i] <= 1)
            prog.AddLinearConstraint(qcom(t)[i] >= -1)
            
            for k in range(num_limbs):
                prog.AddLinearConstraint(qlimb[k](t)[i] <= 1)
                prog.AddLinearConstraint(qlimb[k](t)[i] >= -1)

    for t in ts[:-1]:
        for i in range(dim):
            total_contact_force = sum([contact_force[k](t)[i] for k in range(num_limbs)])
            prog.AddLinearConstraint(total_contact_force + gravity_force[i] == robot.mass * acom(t)[i])

    for i in range(dim):
        prog.AddLinearConstraint(qcom(ts[0])[i] == initial_state.qcom[i])
        prog.AddLinearConstraint(vcom(ts[0])[i].Expand() == initial_state.vcom[i])
        for k in range(num_limbs):
            prog.AddLinearConstraint(qlimb[k](ts[0])[i] == initial_state.qlimb[k][i])
            
    for t in ts[1:]:
        for (k, polytope) in enumerate(robot.limb_bounds):
            A = polytope.getA()
            b = polytope.getB()
            offset = qlimb[k].from_below(t) - qcom.from_below(t)
            for i in range(A.shape[0]):
                prog.AddLinearConstraint((A[i, :].dot(offset) - b[i]).Expand() <= 0)

    all_force_vars = np.hstack([contact_force[k](t) for t in ts[:-1] for k in range(num_limbs)])
    print(all_force_vars.shape)
    prog.AddQuadraticCost(0.01 * np.eye(len(all_force_vars)),
                          np.zeros(len(all_force_vars)),
                          all_force_vars)
    all_qcom_vars = np.hstack([p[0] for p in qcom.functions])
    print(all_qcom_vars.shape)
    prog.AddQuadraticCost(0.01 * np.eye(len(all_qcom_vars)),
                          np.zeros(len(all_qcom_vars)),
                          all_qcom_vars)


    result = prog.Solve()
    print(result)
    assert result == mp.SolutionResult.kSolutionFound

    qcom = get_piecewise_solution(prog, qcom)
    vcom = qcom.map(Polynomial.derivative)
    qlimb = [get_piecewise_solution(prog, qlimb[k]) for k in range(num_limbs)]
    flimb = [get_piecewise_solution(prog, contact_force[k]) for k in range(num_limbs)]
    contact = [get_piecewise_solution(prog, contact[k]) for k in range(num_limbs)]
    return (Trajectory(
        [qcom, vcom] + qlimb,
        lambda qcom, vcom, *qlimb: BoxAtlasState(robot, qcom=qcom, vcom=vcom, qlimb=qlimb)
    ), Trajectory(
        flimb,
        lambda *flimb: BoxAtlasInput(robot, flimb=flimb)
    ), contact)

In [110]:
surfaces = [
    Surface(Polyhedron.fromBounds([1, -1], [1, 1]),
            Polyhedron(np.array([[1, -1], [1, 1]]), np.array([0, 0]))),
    Surface(Polyhedron.fromBounds([-1, -1], [1, -1]), 
            Polyhedron(np.array([[-1, -1], [1, -1]]), np.array([0, 0]))),
    Surface(Polyhedron.fromBounds([-1, -1], [1, -1]), 
            Polyhedron(np.array([[-1, -1], [1, -1]]), np.array([0, 0]))),
    Surface(Polyhedron.fromBounds([-1, -1], [-1, 1]),
            Polyhedron(np.array([[-1, -1], [-1, 1]]), np.array([0, 0]))),
]

env = Environment(surfaces)
atlas = BoxAtlas()
initial_state = BoxAtlasState(atlas)
initial_state.qcom = np.array([0, 0])
initial_state.vcom = np.array([10.0, 0])
initial_state.qlimb = map(np.array, [[0.75, 0], [0.25, -0.75], [-0.25, -0.75], [-0.75, 0]])

states, inputs, zlimb = contact_stabilize(initial_state, env)

(160,)
(40,)
SolutionResult.kSolutionFound


In [113]:
for t in linspace(0, 0.19, 100):
    states(t).draw(vis)
    time.sleep(0.01)

In [55]:
states(0).draw(vis)